In [4]:
import pandas as pd
import plotly.express as px
import os

### Pythia


In [82]:
path = "data/pythia-70-benchmark/"

csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
dfs = [pd.read_csv(os.path.join(path, f)) for f in csv_files]
df = pd.concat(dfs, ignore_index=True)
df.columns

Index(['Name', 'State', 'Notes', 'User', 'Tags', 'Created', 'Runtime', 'Sweep',
       'act', 'batch_size', 'beta1', 'beta2', 'buffer_batches', 'buffer_mult',
       'buffer_size', 'cfg_file', 'd_in', 'data_path',
       'dead_direction_frequency', 'expansion_factor', 'have tried',
       'l1_coeff', 'l1_target', 'layer', 'lr', 'model', 'model_batch_size',
       'num_eval_batches', 'num_eval_prompts', 'num_eval_tokens',
       'num_training_tokens', 'reg', 'reg_coeff_2', 'save_checkpoint_models',
       'save_path', 'seed', 'seq_len', 'to try', 'tried', 'use_wandb', 'wd',
       'W_dec_norm', 'W_enc_norm', 'avg_directions', 'batch', 'bias_kurtosis',
       'bias_mean', 'bias_skew', 'bias_std', 'dead_directions', 'decoder_sim',
       'encoder_sim', 'entropy', 'epoch', 'feature_cosine_sim_mean',
       'feature_cosine_sim_variance', 'hoyer_loss', 'l1_loss', 'loss',
       'loss_recovered', 'mean_feature_unembed_cosine_sim_kurtosis',
       'mse_loss', 'reconstruction_loss', 'reg_loss',

In [83]:
# Update reg column with new names
df["reg"] = df["reg"].map({
    "l1": "L1",
    "pure_sqrt": "Sqrt",
    "combined_hoyer_l1": "Hoyer",
})

In [84]:
df[["layer", "reg", "loss_recovered", "avg_directions"]]

,layer,reg,loss_recovered,avg_directions
0,5,Sqrt,0.927428,25.455315
1,5,L1,0.914093,25.590157
2,4,Sqrt,0.810628,25.279528
3,4,L1,0.793476,25.045275
4,3,Sqrt,0.832672,25.624607
5,3,L1,0.783144,25.660040
6,2,Sqrt,0.841208,25.821260
7,2,L1,0.808678,24.539566
8,1,Sqrt,0.899811,26.380905
9,1,L1,0.876527,26.567913


In [85]:
tmp_df = df[["layer", "reg", "loss_recovered", "avg_directions"]].copy()
tmp_df["loss_recovered"] = tmp_df["loss_recovered"]*100
# round all columns
tmp_df = tmp_df.round(2)
pivot_df = tmp_df.pivot(index="layer", columns='reg').reset_index()
pivot_df.columns = ['Layer', 'Loss Recovered (L0)', 'Loss Recovered (L1)', 'Avg Directions (L0)', 'Avg Directions (L1)']

tsv_output = pivot_df.to_csv(sep='\t')
print(tsv_output)

	Layer	Loss Recovered (L0)	Loss Recovered (L1)	Avg Directions (L0)	Avg Directions (L1)
0	0	99.42	99.64	22.87	24.16
1	1	87.65	89.98	26.57	26.38
2	2	80.87	84.12	24.54	25.82
3	3	78.31	83.27	25.66	25.62
4	4	79.35	81.06	25.05	25.28
5	5	91.41	92.74	25.59	25.46



In [86]:
g = px.scatter(df, x="layer", y="loss_recovered", color="reg")
g.update_layout(
    title="Comparison L0 and L1 regularization on Pythia 70M",
    xaxis_title="Layer",
    yaxis_title="Loss recovered",
    legend_title_text='Regularization',
    width=800
)

#new_names = {"pure_sqrt": "L0", "l1": "L1"}
#g.for_each_trace(lambda t: t.update(name=new_names[t.name]))

## TinyStories

In [88]:
path = "data/tinystories-33m-benchmark/"
csv_files = [f for f in os.listdir(path) if f.endswith('.csv')]
dfs = [pd.read_csv(os.path.join(path, f)) for f in csv_files]
df = pd.concat(dfs, ignore_index=True)
df = df[['reg', 'loss_recovered', 'avg_directions', "seed"]]
df["reg"] = df["reg"].map({
    "l1": "L1",
    "pure_sqrt": "Sqrt",
    "combined_hoyer_l1": "Hoyer",
})
df["loss_recovered_percent"] = df["loss_recovered"]*100

In [89]:
df

,reg,loss_recovered,avg_directions,seed,loss_recovered_percent
0,Sqrt,0.735098,25.065748,49,73.509823
1,Sqrt,0.738870,24.970472,48,73.886994
2,Sqrt,0.735246,25.125000,47,73.524557
3,L1,0.695799,25.981298,49,69.579943
4,Hoyer,0.712018,27.046850,49,71.201751
5,L1,0.694776,25.938583,48,69.477635
6,Hoyer,0.710407,26.766731,48,71.040736
7,L1,0.694026,26.270275,47,69.402576
8,Hoyer,0.711426,26.991142,47,71.142624


In [90]:
pivot_df = df[["seed", "loss_recovered_percent", "avg_directions", "reg"]].pivot(index="seed", columns='reg').reset_index()
pivot_df.round(2)
pivot_df
tsv_output = pivot_df.to_csv(sep='\t')
print(tsv_output)

	seed	loss_recovered_percent	loss_recovered_percent	loss_recovered_percent	avg_directions	avg_directions	avg_directions
reg		Hoyer	L1	Sqrt	Hoyer	L1	Sqrt
0	47	71.14262365841445	69.40257634851308	73.52455731549503	26.99114227294922	26.2702751159668	25.125
1	48	71.04073648343163	69.47763471950599	73.88699409218393	26.76673126220703	25.93858337402344	24.97047233581543
2	49	71.20175113512235	69.57994329215413	73.5098231244271	27.046850204467773	25.981298446655277	25.06574821472168



In [81]:
g = px.scatter(df, x="avg_directions", y="loss_recovered", color="reg")
g.update_layout(
    title="Regularization comparison across 3 seeds",
    xaxis_title="L0",
    yaxis_title="Loss recovered",
    legend_title_text='Regularization',
    width=800
)